# ShorelineMonitor: Change Rate

The ShorelineMonitor dataset provides [Satellite-Derived Shorelines (SDS)](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-shorelines/collection.json) extracted from annually
composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global
view of coastal change and shoreline dynamics, serving as a foundation for coastal
analytics, modeling and management. The shorelines have been mapped onto the [Global Coastal Transect System](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/gcts/collection.json) to form a [new dataset](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-series/collection.json) of more than 7.5 million time-series. 

This notebook shows how to explore multi-decadal trends in shoreline change that are extracted from the full dataset of time series. The dataset is available upon reasonable request. Please contact the data provider for more information or collaboration opportunities.

In [72]:
import os

import numpy as np
import matplotlib.pyplot as plt
import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from coastpy.stac.utils import read_snapshot

# Specify the absolute path to the .env file
dotenv_path = "/Users/juulhemmes/Documents/Studie/Msc/Thesis/coastpy/ATT64895.env"

load_dotenv(dotenv_path=dotenv_path)

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [73]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

,type,stac_version,stac_extensions,id,geometry,bbox,links,assets,collection,created,table:columns,proj:code,proj:bbox,table:row_count,datetime,href,alternate_href
0,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n67e10-c90f1c,"POLYGON ((90.02796 66.50459, 90.02796 81.86757...","[10.415408379310252, 66.50458545284066, 90.027...",[{'href': 'az://gctr/release/2025-03-10/n67e10...,{'data': {'description': ' A Parquet dataset c...,gctr,2025-03-15 15:47:59.740777+00:00,"[{'name': 'transect_id', 'type': 'string'}, {'...",EPSG:4326,"[10.415408379310252, 66.50458545284066, 90.027...",523905,2023-02-09 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,None
1,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n67w180-e6c7b3,"POLYGON ((-89.95719 66.50558, -89.95719 81.939...","[-179.9997518656276, 66.50557986809558, -89.95...",[{'href': 'az://gctr/release/2025-03-10/n67w18...,{'data': {'description': ' A Parquet dataset c...,gctr,2025-03-15 15:48:16.011594+00:00,"[{'name': 'transect_id', 'type': 'string'}, {'...",EPSG:4326,"[-179.9997518656276, 66.50557986809558, -89.95...",650158,2023-02-09 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,None
2,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n67w90-d9edb1,"POLYGON ((-7.90362 66.50436, -7.90362 83.66967...","[-90.05929688568439, 66.50435737903862, -7.903...",[{'href': 'az://gctr/release/2025-03-10/n67w90...,{'data': {'description': ' A Parquet dataset c...,gctr,2025-03-15 15:48:39.736770+00:00,"[{'name': 'transect_id', 'type': 'string'}, {'...",EPSG:4326,"[-90.05929688568439, 66.50435737903862, -7.903...",803314,2023-02-09 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,None
3,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,n69e90-175cd6,"POLYGON ((179.99986 68.75571, 179.99986 81.283...","[89.97127383184208, 68.75571417997578, 179.999...",[{'href': 'az://gctr/release/2025-03-10/n69e90...,{'data': {'description': ' A Parquet dataset c...,gctr,2025-03-15 15:48:53.751665+00:00,"[{'name': 'transect_id', 'type': 'string'}, {'...",EPSG:4326,"[89.97127383184208, 68.75571417997578, 179.999...",262137,2023-02-09 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,None
4,Feature,1.1.0,[https://stac-extensions.github.io/projection/...,s00e90-3c5754,"POLYGON ((179.99973 -0.00884, 179.99973 65.083...","[89.99737841548624, -0.00884311529207947, 179....",[{'href': 'az://gctr/release/2025-03-10/s00e90...,{'data': {'description': ' A Parquet dataset c...,gctr,2025-03-15 15:48:57.699996+00:00,"[{'name': 'transect_id', 'type': 'string'}, {'...",EPSG:4326,"[89.99737841548624, -0.00884311529207947, 179....",1438133,2023-02-09 00:00:00+00:00,https://coclico.blob.core.windows.net/gctr/rel...,None


In [74]:
snapshot.explore()

In [75]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

Map(center=[43.32, -1.97], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [76]:
from coastpy.geo.utils import get_region_of_interest_from_map

# roi = get_region_of_interest_from_map(m, default_extent=(-30, 30, 50, 75))
# west, south, east, north = roi.geometry.item().bounds
west, south, east, north = -120, -56, -35, 33

## Fetch data from the database

In [77]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    columns = ["transect_id", "lon", "lat", "geometry", "continent", "common_country_name", "sds:change_rate"]  # when you don't need all data
)

In [78]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Shape: (1645926, 7)


,transect_id,lon,lat,geometry,continent,common_country_name,sds:change_rate
0,cl05270s03tr01840126,-90.055740,29.379868,"LINESTRING (-90.06473 29.38426, -90.04675 29.3...",NA,United States,-2.504056
1,cl05270s03tr01838526,-90.066757,29.370701,"LINESTRING (-90.07681 29.37263, -90.0567 29.36...",NA,United States,1.159282
2,cl05270s03tr01840326,-90.056099,29.381588,"LINESTRING (-90.06483 29.37682, -90.04736 29.3...",NA,United States,-12.493272
3,cl05270s03tr01840026,-90.056366,29.379175,"LINESTRING (-90.06243 29.38646, -90.0503 29.37...",NA,United States,0.746486
4,cl05270s03tr01839926,-90.057281,29.378796,"LINESTRING (-90.06028 29.38742, -90.05428 29.3...",NA,United States,0.996928


In [71]:
print(df['common_country_name'].unique())

['Mexico' 'Chile' 'Ecuador' 'Guatemala' 'El Salvador' 'Argentina' 'Peru'
 'Cuba' 'Honduras' 'Nicaragua' 'Costa Rica' 'Panama' 'Colombia' 'Belize'
 'Venezuela' 'Haiti' 'Dominican Republic' 'Jamaica' 'Uruguay' 'Brazil'
 'Trinidad and Tobago' 'Suriname' 'Guyana' 'French Guiana']


In [61]:
# Define a list of countries in South and Central America
south_and_central_america_countries = [
    "Mexico", "Guatemala", "El Salvador", "Ecuador", "Cuba", "Costa Rica", "Panama", 
    "Honduras", "Belize", "Nicaragua", "French Guiana", "Colombia", "Venezuela", 
    "Dominican Republic", "Haiti", "Suriname", "Guyana", "Brazil", "Jamaica", 
    "Trinidad and Tobago", "Chile", "Peru", "Argentina", "Uruguay", "Paraguay", 
    "Bolivia"
]

# Apply the filter to keep only these countries
df_latin_america = df[df["common_country_name"].isin(south_and_central_america_countries)]


In [62]:
ac = gpd.GeoDataFrame(
    df_latin_america[["transect_id", "sds:change_rate"]],
    geometry=gpd.GeoSeries.from_xy(df.lon, df.lat, crs=4326),
)

In [63]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

df = df[df["common_country_name"].isin(south_and_central_america_countries)]   # Filter to Europe only

# Step 1: Sort dataset by transect ID to ensure consecutive order
df = df.sort_values(by=["transect_id"]).reset_index(drop=True)

# Step 2: Identify hotspots (50+ consecutive transects with the same trend exceeding ±0.5 m/y)
hotspot_lists = []  # Stores lists of consecutive transect IDs
current_hotspot = []  # Temporary storage for an ongoing hotspot group
current_trend = None  # Tracks whether we're in an erosion or accretion hotspot

for i in range(len(df)):
    transect_id = df.iloc[i]["transect_id"]
    change_rate = df.iloc[i]["sds:change_rate"]

    # Determine trend based on change rate
    if change_rate < -0.5:
        trend = "erosion"
    elif change_rate > 0.5:
        trend = "accretion"
    else:
        trend = None  # This is now allowed but won’t be considered a hotspot.

    if trend is None:  # If no strong trend, break the hotspot
        if len(current_hotspot) >= 21:  # Only keep valid hotspots
            hotspot_lists.append(current_hotspot)
        current_hotspot = []  # Reset for next potential hotspot
        current_trend = None
    elif current_trend is None:  # Start first hotspot
        current_hotspot.append(transect_id)
        current_trend = trend
    elif trend == current_trend:  # Continue hotspot if trend remains the same
        current_hotspot.append(transect_id)
    else:  # If trend switches, store the hotspot and start a new one
        if len(current_hotspot) >= 51:  # Only store valid hotspots
            hotspot_lists.append(current_hotspot)
        current_hotspot = [transect_id]  # Start a new hotspot
        current_trend = trend

# Append the last detected hotspot if it meets the threshold
if len(current_hotspot) >= 51:
    hotspot_lists.append(current_hotspot)

# # Step 3: Print hotspot lengths for investigation
# hotspot_lengths = [len(hotspot) for hotspot in hotspot_lists]
# print("Hotspot Length Counts:", hotspot_lengths)

# # Step 4: Count occurrences of different hotspot sizes
# hotspot_length_counts = Counter(hotspot_lengths)
# print("\nSummary of Hotspot Sizes:")
# for length, count in sorted(hotspot_length_counts.items()):
#     print(f"Hotspots of length {length}: {count}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Shape: (1645926, 7)


In [64]:
print(len(hotspot_lists))

2292


In [65]:
from shapely.geometry import Polygon

# Identify hotspot statistics
hotspot_data = []

for hotspot in hotspot_lists:
    hotspot_df = df[df["transect_id"].isin(hotspot)]

    lat_center = hotspot_df["lat"].median()  # Central latitude
    lon_center = hotspot_df["lon"].median()  # Central longitude
    min_lat, max_lat = hotspot_df["lat"].min(), hotspot_df["lat"].max()
    min_lon, max_lon = hotspot_df["lon"].min(), hotspot_df["lon"].max()
    
    min_rate = hotspot_df["sds:change_rate"].min()  # Most severe erosion
    max_rate = hotspot_df["sds:change_rate"].max()  # Most extreme accretion
    mean_rate = hotspot_df["sds:change_rate"].mean()
    median_rate = hotspot_df["sds:change_rate"].median()

    # Compute hotspot length (100m per transect)
    hotspot_length_m = (len(hotspot) - 1) * 100  # Convert to meters
    hotspot_length_km = hotspot_length_m / 1000  # Convert to kilometers

    # Determine if it's an erosion or accretion hotspot
    trend_type = "erosion" if abs(min_rate) > max_rate else "accretion"
    max_abs_rate = max(abs(min_rate), abs(max_rate))  # Max absolute rate
    
    # Classify by rate bins
    if 0.5 <= max_abs_rate < 2:
        size_category = 50
    elif 2 <= max_abs_rate < 5:
        size_category = 100
    elif 5 <= max_abs_rate < 10:
        size_category = 200
    elif max_abs_rate >= 20:
        size_category = 400
    else:
        size_category = 10  # Default small size if unclear

    # Extract country name from dataset
    country_name = hotspot_df["common_country_name"].mode()[0]  # Most common country in the stretch

    # Create bounding box polygon from min/max lat/lon
    hotspot_polygon = Polygon([
        (min_lon, min_lat),  # Bottom-left
        (max_lon, min_lat),  # Bottom-right
        (max_lon, max_lat),  # Top-right
        (min_lon, max_lat),  # Top-left
        (min_lon, min_lat)   # Close the polygon
    ])

    # Store data for table
    hotspot_data.append({
        "lat": lat_center,
        "lon": lon_center,
        "min_lat": min_lat,
        "max_lat": max_lat,
        "min_lon": min_lon,
        "max_lon": max_lon,
        "trend": trend_type,
        "mean_rate": mean_rate,
        "min_rate": min_rate,
        "max_rate": max_rate,
        "median_rate": median_rate,
        "size_category": size_category,
        "hotspot_length_km": hotspot_length_km,
        "country": country_name,
        "geometry": hotspot_polygon  # Store polygon
    })

# Convert to GeoDataFrame
hotspot_gdf = gpd.GeoDataFrame(hotspot_data, geometry="geometry", crs="EPSG:4326")



In [66]:
hotspot_gdf = hotspot_gdf.sort_values(by=["hotspot_length_km"]).reset_index(drop=True)

In [67]:
hotspot_gdf


,lat,lon,min_lat,max_lat,min_lon,max_lon,trend,mean_rate,min_rate,max_rate,median_rate,size_category,hotspot_length_km,country,geometry
0,5.915023,-56.460052,5.913947,5.917288,-56.468788,-56.451088,erosion,-5.692541,-14.342649,-1.461009,-5.158549,10,2.0,Suriname,"POLYGON ((-56.46879 5.91395, -56.45109 5.91395..."
1,21.463177,-78.594841,21.455425,21.470581,-78.600327,-78.589935,erosion,-0.586758,-0.720864,-0.505979,-0.569088,50,2.0,Cuba,"POLYGON ((-78.60033 21.45543, -78.58994 21.455..."
2,21.914520,-84.926826,21.907316,21.916830,-84.932594,-84.918137,erosion,-1.187092,-2.366547,-0.522041,-0.940376,100,2.0,Cuba,"POLYGON ((-84.93259 21.90732, -84.91814 21.907..."
3,-3.270641,-80.017433,-3.277085,-3.262614,-80.023720,-80.013611,erosion,-1.535818,-2.652203,-0.596454,-1.517828,100,2.0,Ecuador,"POLYGON ((-80.02372 -3.27708, -80.01361 -3.277..."
4,-2.491586,-79.980980,-2.496030,-2.490378,-79.989838,-79.973236,accretion,0.831091,0.617882,1.130084,0.809929,50,2.0,Ecuador,"POLYGON ((-79.98984 -2.49603, -79.97324 -2.496..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,-24.787483,-47.606995,-24.866026,-24.719250,-47.713486,-47.492466,accretion,1.880403,0.508214,4.307619,1.890463,100,27.7,Brazil,"POLYGON ((-47.71349 -24.86603, -47.49247 -24.8..."
2288,13.900037,-90.519470,13.856936,13.925393,-90.649529,-90.394386,accretion,1.416285,0.579456,4.088545,1.289473,100,28.7,Guatemala,"POLYGON ((-90.64953 13.85694, -90.39439 13.856..."
2289,1.787013,-50.105148,1.725263,1.813282,-50.248795,-49.969536,erosion,-13.959206,-21.828833,-1.282221,-13.598989,400,33.2,Brazil,"POLYGON ((-50.24879 1.72526, -49.96954 1.72526..."
2290,3.227351,-51.044632,3.086943,3.368175,-51.091381,-51.017006,erosion,-17.740763,-37.152481,-1.530342,-18.601387,400,33.3,Brazil,"POLYGON ((-51.09138 3.08694, -51.01701 3.08694..."


In [68]:
# Find the longest hotspot
longest_hotspot = hotspot_gdf.loc[hotspot_gdf["hotspot_length_km"].idxmax()]
print(f"🔹 Longest Hotspot: {longest_hotspot['hotspot_length_km']:.2f} km")
print(f"   📍 Country: {longest_hotspot['country']}")
print(f"   🔴 Min Erosion Rate: {longest_hotspot['min_rate']:.2f} m/year")
print(f"   🟢 Max Accretion Rate: {longest_hotspot['max_rate']:.2f} m/year\n")

# Find the hotspot with the most severe erosion
highest_erosion_hotspot = hotspot_gdf.loc[hotspot_gdf["min_rate"].idxmin()]
print(f"⚠️ Most Severe Erosion: {highest_erosion_hotspot['min_rate']:.2f} m/year")
print(f"   📍 Country: {highest_erosion_hotspot['country']}")
print(f"   🌍 Location: ({highest_erosion_hotspot['lat']:.2f}, {highest_erosion_hotspot['lon']:.2f})\n")

# Find the hotspot with the highest accretion rate
highest_accretion_hotspot = hotspot_gdf.loc[hotspot_gdf["max_rate"].idxmax()]
print(f"✅ Highest Accretion: {highest_accretion_hotspot['max_rate']:.2f} m/year")
print(f"   📍 Country: {highest_accretion_hotspot['country']}")
print(f"   🌍 Location: ({highest_accretion_hotspot['lat']:.2f}, {highest_accretion_hotspot['lon']:.2f})\n")

# Country-level summary: Number of hotspots per country
country_counts = hotspot_gdf["country"].value_counts()
print("🌍 Number of Hotspots Per Country:")
print(country_counts.to_string(), "\n")

# Average hotspot length per country
country_avg_length = hotspot_gdf.groupby("country")["hotspot_length_km"].mean().sort_values(ascending=False)
print("📏 Average Hotspot Length Per Country (km):")
print(country_avg_length.to_string(), "\n")

# Country with the longest average hotspots
longest_avg_hotspot_country = country_avg_length.idxmax()
print(f"🏆 Country with Longest Average Hotspots: {longest_avg_hotspot_country} ({country_avg_length.max():.2f} km)\n")

# Mean and median erosion/accretion rates per country
country_stats = hotspot_gdf.groupby("country").agg(
    mean_erosion=("min_rate", "mean"),
    max_erosion=("min_rate", "min"),
    mean_accretion=("max_rate", "mean"),
    max_accretion=("max_rate", "max")
).sort_values(by="max_erosion", ascending=True)  # Sort by most severe erosion

print("📉 Most Severe Erosion & Accretion by Country:")
print(country_stats.to_string(), "\n")

# Most balanced country (where mean erosion and accretion are closest)
balanced_country = (country_stats["mean_erosion"] + country_stats["mean_accretion"]).abs().idxmin()
print(f"⚖️ Most Balanced Shoreline Change: {balanced_country} (Erosion: {country_stats.loc[balanced_country, 'mean_erosion']:.2f}, Accretion: {country_stats.loc[balanced_country, 'mean_accretion']:.2f})\n")

🔹 Longest Hotspot: 39.70 km
   📍 Country: Venezuela
   🔴 Min Erosion Rate: -9.70 m/year
   🟢 Max Accretion Rate: -1.01 m/year

⚠️ Most Severe Erosion: -113.08 m/year
   📍 Country: French Guiana
   🌍 Location: (5.67, -53.71)

✅ Highest Accretion: 85.02 m/year
   📍 Country: French Guiana
   🌍 Location: (5.57, -53.46)

🌍 Number of Hotspots Per Country:
country
Brazil                 593
Mexico                 434
Venezuela              173
Peru                   171
Chile                  167
Argentina              164
Colombia               109
Cuba                    71
Ecuador                 63
Honduras                54
Nicaragua               48
Panama                  45
Costa Rica              35
Guyana                  27
Guatemala               27
Haiti                   18
Dominican Republic      17
El Salvador             16
French Guiana           14
Suriname                14
Uruguay                 13
Jamaica                  8
Belize                   6
Trinidad and Tobago

In [69]:
# Compute total eroding and accreting coastline lengths per country
country_length_stats = hotspot_gdf.groupby(["country", "trend"])["hotspot_length_km"].sum().unstack(fill_value=0)

# Rename columns for clarity
country_length_stats.columns = ["Total Accreting Length (km)", "Total Eroding Length (km)"]

# Sort by longest eroding and accreting coastlines
longest_erosion_countries = country_length_stats.sort_values("Total Eroding Length (km)", ascending=False)
longest_accretion_countries = country_length_stats.sort_values("Total Accreting Length (km)", ascending=False)

# Print longest eroding coastlines
print("🌊 Countries with the Longest Total Eroding Coastlines (km):")
print(longest_erosion_countries["Total Eroding Length (km)"].to_string(), "\n")

# Print longest accreting coastlines
print("🏝️ Countries with the Longest Total Accreting Coastlines (km):")
print(longest_accretion_countries["Total Accreting Length (km)"].to_string(), "\n")

🌊 Countries with the Longest Total Eroding Coastlines (km):
country
Brazil                 1932.1
Mexico                  954.6
Argentina               499.8
Venezuela               459.3
Colombia                277.3
Cuba                    197.0
Nicaragua               173.0
Chile                   141.8
Peru                     91.3
Ecuador                  91.2
Guyana                   89.2
Panama                   82.8
French Guiana            68.0
Suriname                 60.4
Honduras                 59.1
Haiti                    29.9
El Salvador              25.4
Dominican Republic       22.5
Jamaica                  18.9
Belize                   17.6
Costa Rica               17.6
Uruguay                  14.5
Guatemala                14.0
Trinidad and Tobago      11.6 

🏝️ Countries with the Longest Total Accreting Coastlines (km):
country
Mexico                 1218.1
Brazil                  853.8
Peru                    770.7
Chile                   418.2
Venezuela          

In [ ]:
import folium

# Define center of Europe for initial view
map_center = [50, 10]  # Latitude, Longitude
m_folium = folium.Map(location=map_center, zoom_start=4, tiles="OpenStreetMap")

# Define function for correct sizing
def classify_size(trend, min_rate, max_rate):
    abs_rate = abs(min_rate) if trend == "erosion" else abs(max_rate)  # Correct scaling
    if 0.5 <= abs_rate < 2:
        return 5
    elif 2 <= abs_rate < 5:
        return 10
    elif 5 <= abs_rate < 10:
        return 15
    elif abs_rate >= 20:
        return 25
    return 3  # Default small size

# Add hotspots to the map
for _, row in hotspot_gdf.iterrows():
    color = "red" if row["trend"] == "erosion" else "green"  # Red for erosion, green for accretion
    size = classify_size(row["trend"], row["min_rate"], row["max_rate"])  # Correct size logic

    popup_content = f"""
    <b>Hotspot Info</b><br>
    <b>Trend:</b> {row['trend'].capitalize()}<br>
    <b>Mean Rate:</b> {row['mean_rate']:.2f} m/yr<br>
    <b>Min Rate:</b> {row['min_rate']:.2f} m/yr<br>
    <b>Max Rate:</b> {row['max_rate']:.2f} m/yr<br>
    <b>Median Rate:</b> {row['median_rate']:.2f} m/yr
    """

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=size,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m_folium)

# Add a legend manually using HTML and CSS
legend_html = """
<div style="
    position: fixed; 
    bottom: 20px; left: 20px; 
    background-color: white; 
    padding: 10px; 
    border-radius: 5px; 
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3); 
    font-size: 14px;
    line-height: 1.6;
    width: 180px;
    z-index:9999;
">
    <b>Erosion/Accretion Rates</b><br>
    <div style="display: flex; align-items: center;">
        <div style="width: 12px; height: 12px; background: gray; border-radius: 50%; margin-right: 8px;"></div> 0.5 - 2 m/yr
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 16px; height: 16px; background: gray; border-radius: 50%; margin-right: 8px;"></div> 2 - 5 m/yr
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background: gray; border-radius: 50%; margin-right: 8px;"></div> 5 - 10 m/yr
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 25px; height: 25px; background: gray; border-radius: 50%; margin-right: 8px;"></div> > 20 m/yr
    </div>
</div>
"""

m_folium.get_root().html.add_child(folium.Element(legend_html))

# Save map as an interactive HTML file
map_filename = "shoreline_hotspots_latin_and_central_america.html"
m_folium.save(map_filename)

print(f"✅ Map successfully saved as {map_filename}. Open this file in a browser to view it.")


✅ Map successfully saved as shoreline_hotspots_latin_and_central_america.html. Open this file in a browser to view it.
